# Spark assignment 1: Pairs

Find all the pairs of two consequent words where the first word is “narodnaya”. For each pair, count the number of occurrences in the Wikipedia dump. Print all the pairs with their count in a lexicographical order. Output format is “word_pair <tab> count”, for example:

*red_apple 100500*

*crazy_zoo 42*

Note that two words in a pair are concatenated with the underscore character, and the result is in the lowercase.

One motivation for counting these continuations is to get a better understanding of the language. Some words, like “the”, have a lot of continuations, while others, like “San”, have just a few (“San Francisco”, for example). One can build a language model with these statistics. If you are interested to learn more, search for “n-gram language model” in the Internet.

In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[*]"))

In [2]:
import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
    except ValueError as e:
        return []
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    return words

In [3]:
def make_bigrams_and_filter(words, start_word):
    result = list()
    
    if not isinstance(words, list):
        return
    
    for i in range(0, len(words)):
        if words[i] != start_word:
            continue
        result.append((words[i] + '_' + words[i + 1], 1))
    
    return result

In [4]:
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 4).map(parse_article)

wiki_lower = wiki.map(lambda words: [x.lower() for x in words])

filtered_bigrams = wiki_lower.flatMap(lambda item: make_bigrams_and_filter(item, 'narodnaya'))

result = filtered_bigrams.reduceByKey(lambda accum, n: accum + n).sortByKey()

for (key, value) in result.collect():
    print "%s\t%d" % (key, value)

narodnaya_gazeta	1
narodnaya_volya	9
